# Clasificación de sonidos urbanos usando Deep Learning

## Análisis de los datos

UrbanSound dataset

Se utiliza un dataset llamado "Urbansound8K", perteneciente a https://urbansounddataset.weebly.com/. El dataset contiene 8732 sound excerpts (<=4s) de diferentes eventos sonoros urbanos divididos en 10 classes:

- Air Conditioner
- Car Horn
- Children Playing
- Dog bark
- Drilling
- Engine Idling
- Gun Shot
- Jackhammer
- Siren
- Street Music

La metadata incluida en el .csv contiene un ID para cada sound excerpt en conjunto con el nombre de su clase.

Todos los archivos están en formato .wav. De todas maneras, el sampling rate, bit depth y número de canales puede variar entre los archivos.

## Ejemplo

Se importan y escuchan algunos de los ejemplos de los audios.

In [5]:
import IPython.display as ipd

ipd.Audio ('../Jupyter Notebooks/Audio Samples/7061-6-0-0.wav')

In [2]:
ipd.Audio ('../Jupyter Notebooks/Audio Samples/22962-4-0-1.wav')

In [3]:
ipd.Audio ('../Jupyter Notebooks/Audio Samples/24074-1-0-4.wav')

In [4]:
ipd.Audio ('../Jupyter Notebooks/Audio Samples/31323-3-0-22.wav')

## Metadata

Leo la metadata incluida en el archivo .csv del dataset

In [5]:
import pandas as pd
metadata = pd.read_csv('../Jupyter Notebooks/Metadata/UrbanSound8K.csv')

#Veo las primeras 5 líneas del csv
metadata.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class_name
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


Puedo analizar cuántos samples tengo de cada clase

In [6]:
samples_per_class = metadata.class_name.value_counts()

print(samples_per_class)

street_music        1000
children_playing    1000
jackhammer          1000
drilling            1000
engine_idling       1000
air_conditioner     1000
dog_bark            1000
siren                929
car_horn             429
gun_shot             374
Name: class_name, dtype: int64


Puedo ver que la mayoría de las clases tienen 1000 samples pero hay 3 clases que tienen menos de 1000. Esto debe ser en cuenta posteriormente para la clasificación.

## Análisis de los parámetros de los archivos de audio: Bit-depth, Channels y Sample Rate

In [8]:
#Voy a iterar a través de cada audio para obtener número de canales, bit-depth y sample rate. Luego, se va a 
#storear en un pandas dataframe

import os
import librosa as lb
from wav_format_processor import WavFormatProcessor

wavformatprocessor = WavFormatProcessor()

audiodata = []

for index, row in metadata.iterrows():
    
    file_name = os.path.join(os.path.abspath('../UrbanSound8K/audio/'),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
    data = wavformatprocessor.wav_file_properties(file_name)
    audiodata.append(data) #appendo en audiodata
    
#Convierto la data a un pandas dataframe

audio_dataframe = pd.DataFrame(audiodata, columns=['num_channels', 'sample_rate', 'bit_depth'])

## Número de canales 

Chequeo el número de canales presente en los archivos de mi dataset.

In [15]:
n_canales = audio_dataframe.num_channels.value_counts(normalize=True)

print (n_canales)

2    0.915369
1    0.084631
Name: num_channels, dtype: float64


Puedo ver que el 91.5% de mis audios son stereo, mientras que el 8.5% son mono.

## Frecuencia de sampleo

La frecuencia de sampleo más común debería ser 44.1kHz o 48kHz. Estos datos me van a servir para determinar si puedo downsamplear o upsamplear mi información.

In [16]:
f_sampleo = audio_dataframe.sample_rate.value_counts(normalize=True)
print(f_sampleo)

44100     0.614979
48000     0.286532
96000     0.069858
24000     0.009391
16000     0.005153
22050     0.005039
11025     0.004466
192000    0.001947
8000      0.001374
11024     0.000802
32000     0.000458
Name: sample_rate, dtype: float64


Sorpresivamente, hay una gran dispersión en los sample rates utilizados en cada audio, aunque la mayoría son de 44.1 kHz o 48 kHz. Esto será una decisión a tomar luego en la etapa de unificación de parámetros.

## Profundidad de bits

Analizo la profundidad de bits de cada archivo de audio. Comúnmente se utilizan 16 o 32 bits, aunque seguramente existan casos de 24 bits.

In [17]:
prof_bits = audio_dataframe.bit_depth.value_counts(normalize=True)
print (prof_bits)

16    0.659414
24    0.315277
32    0.019354
8     0.004924
4     0.001031
Name: bit_depth, dtype: float64
